In [ ]:
import pandas as pd
from my_libs import lib_tools as pt

run_gridSearchCV = True  # True to run hyperparameters optimization with GridSearchCV()
run_optuna = True        # True to run hyperparameters optimization with Optuna

# run_type = 'very-light'
run_type = 'light'
# run_type = 'full'

if run_type == 'very-light': filename_train, filename_test = 'df-very-light-train.pkl', 'df-very-light-test.pkl'
if run_type == 'light'     : filename_train, filename_test = 'df-light-train.pkl', 'df-light-test.pkl'
if run_type == 'full'      : filename_train, filename_test = 'df-full-train.pkl', 'df-full-test.pkl'

# classifier_name = 'DecisionTreeClassifier'
# classifier_name = 'RandomForestClassifier'
# classifier_name = 'GradientBoostingClassifier'

columns = ['catv', 'agg', 'dep', 'col', 'catr', 'catu', 'trajet', 'locp', 'circ', 'situ', 'lum', 'age_cls']
X_train, y_train, X_test, y_test, X_test_final, y_test_final = pt.get_train_valid_test_data(filename_train, filename_test, columns)

In [ ]:
from imblearn.over_sampling import SMOTEN
import time

sampler = SMOTEN()

start_time = time.time()
X_train, y_train = sampler.fit_resample(X_train, y_train)
print(f"X_train : {X_train.shape} - y_train : {y_train.shape}")

X_train['dep'] = X_train['dep'].astype('int')

print(f"--- Smote applied in %s seconds ---" % (time.time() - start_time))

In [ ]:
from my_libs.model_evaluator import ModelEvaluator

params = {'iterations': 50, 'learning_rate': 0.0991, 'random_seed': 123, 'cat_features': list(X_train.columns)}
evaluator = ModelEvaluator(model_type='CatBoostClassifier', params=params, X_train=X_train, y_train=y_train, X_test=X_test_final, y_test=y_test_final)
model = evaluator.evaluate()

In [ ]:
from shapash import SmartExplainer
import pickle
import shap
shap.initjs()

xpl = SmartExplainer(
    model=model,
    # features_dict=house_dict,  # Optional parameter
    # preprocessing=encoder, # Optional: compile step can use inverse_transform method
    # postprocessing=postprocess # Optional: see tutorial postprocessing
)

y_test_final.index = X_test_final.index

xpl.compile(
    x=X_test_final,
    # y_pred=y_pred, # Optional: for your own prediction (by default: model.predict)
    y_target=y_test_final, # Optional: allows to display True Values vs Predicted Values
)

app = xpl.run_app()

In [ ]:
# app.kill()